In [1]:
from AOC_utils import get_day
import numpy as np

day = 24
input_data = get_day(day)

# print out first few lines to get a feel for the data
input_data[:7]

Downloaded day 24 input


['380596900441035, 475034410013298, 238677466991589 @ -141, -244, 154',
 '233796913851006, 262774170759556, 265925724673108 @ 54, 10, 23',
 '276006064958748, 296055609314709, 391999646036593 @ 14, 21, 24',
 '293483465377495, 295344217447825, 424521127946914 @ -5, 21, -13',
 '293267724517878, 460174630326339, 283123920222676 @ -56, -349, 17',
 '109629738653787, 228051835888962, 65687603071905 @ 261, 86, 408',
 '439102647267747, 292512150779892, 283500442341031 @ -257, -11, 67']

In [2]:
example = '''
19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3
'''.split('\n')[1:-1]

In [3]:
len(input_data)

300

In [10]:
import shapely

In [80]:
def parse_data(data, ignore_z=True, test_area_min=200000000000000, test_area_max=400000000000000):

    hailstone_positions = [[int(x) for x in line.split('@')[0].replace(' ', '').split(',')] for line in data]
    hailstone_velocities = [[int(x) for x in line.split('@')[1].replace(' ', '').split(',')] for line in data]
    hailstones = np.array([hailstone_positions, hailstone_velocities]).transpose(1, 0, 2)
    print(hailstones.shape)

    # remove hailstones that are out of bounds and moving away from the test area
    new_hailstones = []
    for hailstone in hailstones:
        coord_positions, coord_velocitys = hailstone
        if ignore_z:
            coord_positions = coord_positions[:2]
            coord_velocitys = coord_velocitys[:2]
        passed = True
        for i in range(len(coord_positions)):
            position = coord_positions[i]
            velocity = coord_velocitys[i]
            if velocity > 0 and position > test_area_max or velocity < 0 and position < test_area_min:
                passed = False
                break
        if passed:
            new_hailstones.append(hailstone)
    hailstones = np.array(new_hailstones).astype(np.float64)
    print(hailstones.shape)
    
    def get_line_ends(hailstone):
        coord_positions, coord_velocitys = hailstone

        line_ends = []
        for i in range(len(coord_positions)):
            position = coord_positions[i]
            velocity = coord_velocitys[i]
            # line_end = position + (velocity * t)
            if velocity > 0:
                test_edge = test_area_max
            else:
                test_edge = test_area_min
            t = (test_edge - position) / velocity
            line_end = [position + (velocity * t) for position, velocity in zip(coord_positions, coord_velocitys)]
            line_ends.append(line_end)

        # find which line end is not out of bounds
        for i in range(len(line_ends)):
            line_end = line_ends[i]
            if ignore_z:
                if all(test_area_min <= np.around(x) <= test_area_max for x in line_end[:2]):
                    return line_end
            else:
                if all(test_area_min <= np.around(x) <= test_area_max for x in line_end):
                    return line_end # THIS COULD BE A PROBLEM
                

        print("no line end found")
        print(coord_positions, coord_velocitys)
        print(line_ends)
            
    hailstone_line_ends = np.array([get_line_ends(hailstone) for hailstone in hailstones])

    # print(hailstone_line_ends.shape)

    lines = []
    for i in range(len(hailstones)):
        hailstone_pos = hailstones[i][0]
        line_end = hailstone_line_ends[i]

        # if line_end is None:
        #     continue

        if ignore_z:
            hailstone_pos = hailstone_pos[:2]
            line_end = line_end[:2]
        lines.append(shapely.LineString([hailstone_pos, line_end]))

    # count the number of intersections
    total = 0
    for i in range(len(lines)):
        line = lines[i]
        for j in range(i+1, len(lines)):
            other_line = lines[j]
            intersection = line.intersection(other_line)
            if type(intersection) != shapely.geometry.point.Point:
                continue
            # print(len(intersection.parameters))
            # print(intersection.x, intersection.y)
            if all(test_area_min <= np.around(x) <= test_area_max for x in [intersection.x, intersection.y]):
                total += 1

            # if line.intersects(other_line):
            #     total += 1

    print("part 1:", total)

In [81]:
parse_data(input_data)

(300, 2, 3)
(300, 2, 3)
part 1: 31208


In [ ]:
31256

In [62]:
parse_data(example, test_area_min=7, test_area_max=27)

(5, 2, 3)
(5, 2, 3)
part 1: 2


In [ ]:
431310784597482, 400000000000000